# Crease Detection in Trimesh

This Jupyter notebook demonstrates how to detect and visualize crease edges (sharp edges) in a mesh using the `trimesh` and `pyrender` libraries.

In [ ]:
# 1. Load your OBJ without processing (to preserve UVs)
loaded = trimesh.load(
    '/home/jeans/progressive_img2sketch/resources/LOD_data_50/28/lod2.obj',
    process=False
)

loaded.show()

In [ ]:
import trimesh
import numpy as np
from trimesh.visual import ColorVisuals
from trimesh.scene.lighting import DirectionalLight, PointLight

# ─── User Options ─────────────────────────────────────────────────────────────
show_mesh         = True
threshold_degrees = 5.0
angle_thresh      = np.deg2rad(threshold_degrees)
# ────────────────────────────────────────────────────────────────────────────────

# 1. Load your OBJ without processing (to preserve UVs)
loaded = trimesh.load(
    '/home/jeans/progressive_img2sketch/resources/LOD_data_50/28/lod1.obj',
    process=False
)

# 2. Extract mesh parts
textured_meshes = (list(loaded.geometry.values())
                   if isinstance(loaded, trimesh.Scene)
                   else [loaded])

# 3. Create scene and add your mesh (textured or white occluder)
scene = trimesh.Scene()
for mesh in textured_meshes:
    if show_mesh:
        scene.add_geometry(mesh)
    else:
        wm = mesh.copy()
        fc = np.tile([255,255,255,255], (len(wm.faces),1))
        wm.visual = ColorVisuals(mesh=wm, face_colors=fc)
        scene.add_geometry(wm)

# 4. Build a welded mesh for crease detection
clean_parts = []
for mesh in textured_meshes:
    clean_parts.append(trimesh.Trimesh(
        vertices=mesh.vertices.copy(),
        faces   =mesh.faces.copy(),
        process =True
    ))
clean_full = trimesh.util.concatenate(clean_parts)

# 5. Sharp‐edge test
fa       = clean_full.face_adjacency_angles     # θ = arccos(n·n)
mask     = fa > angle_thresh
edges    = clean_full.face_adjacency_edges[mask]
segments = clean_full.vertices[edges]

crease = trimesh.load_path(segments)
crease.colors = np.tile([0,0,0,255], (len(crease.entities),1))
scene.add_geometry(crease)

# ─── Render ──────────────────────────────────────────────────────────────────
scene.show()


In [ ]:
import trimesh
import numpy as np
import pyrender
from trimesh.visual import ColorVisuals
import os

os.environ['LIBGL_ALWAYS_SOFTWARE'] = '1'  # force software rendering for pyrender

import trimesh
import numpy as np
import pyrender
from trimesh.visual import ColorVisuals

# ─── User Options ─────────────────────────────────────────────────────────────
show_textured    = True       # False → white occluder, True → textured mesh underneath
threshold_degrees= 5.0
angle_thresh     = np.deg2rad(threshold_degrees)
tube_radius      = 0.02       # radius of each tube
tube_sections    = 8          # subdivisions around the tube
# ────────────────────────────────────────────────────────────────────────────────

# 1. Load your OBJ as a Scene (keeps UVs & materials)
loaded = trimesh.load(
    '/home/jeans/progressive_img2sketch/resources/LOD_data_50/29/lod2.obj',
    process=False
)
textured_meshes = (list(loaded.geometry.values())
                   if isinstance(loaded, trimesh.Scene)
                   else [loaded])

# 2. Build a “clean” welded mesh for adjacency
combined   = trimesh.util.concatenate(textured_meshes)
clean_full = trimesh.Trimesh(
    vertices=combined.vertices,
    faces   =combined.faces,
    process =True
)

# 3. Detect sharp edges (angle between normals)
fa       = clean_full.face_adjacency_angles       # θ = arccos(n₁·n₂)
mask     = fa > angle_thresh
edges    = clean_full.face_adjacency_edges[mask]  # (K,2)
segments = clean_full.vertices[edges]             # (K,2,3)

# 4. Set up pyrender scene
scene = pyrender.Scene(ambient_light=[0.3,0.3,0.3,1.0])

# 5. Add mesh(es) first (textured or white occluder)
for mesh in textured_meshes:
    if show_textured:
        pm = pyrender.Mesh.from_trimesh(mesh, smooth=False)
    else:
        wm = mesh.copy()
        fc = np.tile([255,255,255,255], (len(wm.faces), 1))
        wm.visual = ColorVisuals(mesh=wm, face_colors=fc)
        pm = pyrender.Mesh.from_trimesh(wm, smooth=False)
    scene.add(pm)

# 6. Material for the crease tubes (black)
line_mat = pyrender.MetallicRoughnessMaterial(
    baseColorFactor=[0,0,0,1],
    metallicFactor=0.0,
    roughnessFactor=0.9
)

# 7. Build & add a tube for each segment
for p0, p1 in segments:
    v = p1 - p0
    L = np.linalg.norm(v)
    if L < 1e-6:
        continue
    # cylinder along +Z at origin
    cyl = trimesh.creation.cylinder(
        radius=tube_radius,
        height=L,
        sections=tube_sections
    )
    # move base to origin
    cyl.apply_translation([0, 0, L/2.0])
    # align +Z to our segment direction
    transform = trimesh.geometry.align_vectors([0,0,1], v / L)
    cyl.apply_transform(transform)
    # translate into place
    cyl.apply_translation(p0)
    # add to scene
    tube = pyrender.Mesh.from_trimesh(cyl, material=line_mat)
    scene.add(tube)

# 8. Show in pyrender
pyrender.Viewer(scene, use_raymond_lighting=True)
